In [1]:
import pandas as pd
from sklearn.manifold import TSNE
import networkx as nx
import numpy as np
from node2vec import Node2Vec
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN

In [2]:
df = pd.read_csv('Iron_dealers_data.csv')
df.head()

,Seller ID,Buyer ID,Value
0,1309,1011,1225513.0
1,1309,1011,1179061.0
2,1309,1011,1119561.0
3,1309,1011,1200934.0
4,1309,1011,1658957.0


In [3]:
G = nx.MultiDiGraph()

for index, row in df.iterrows():
    G.add_edge(int(row['Seller ID']), int(row['Buyer ID']), weight=row['Value'])

In [4]:
edges = []

# Computing weights of the undirected edges
for u in G.nodes():
    for v in G.successors(u):
        weight = 0
        r = np.zeros(4)
        w = np.array([2,3,3,4])
        for t in G.successors(v):
            if G.has_edge(t,u):
                # Found a 3 cycle
                reciprocal = 0
                if G.has_edge(v,u):
                    reciprocal+=1
                if G.has_edge(t,v):
                    reciprocal+=1
                if G.has_edge(u,t):
                    reciprocal+=1
                
                r[reciprocal]+=1
                weight = max(weight,np.max(w*r))

        edges.append([u,v,weight])

In [5]:
# Constructing the undirected graph
UG = nx.Graph()
for edge in edges:
    u,v,w = edge[0],edge[1],edge[2]
    if UG.has_edge(u,v):
        UG[u][v]['weight'] = max(UG[u][v]['weight'],w)
    else:
        UG.add_edge(u,v,weight=w)

In [6]:
model = Node2Vec(UG,dimensions=20,walk_length=30,num_walks=400,workers=6)
embeddings = model.fit(window = 10,min_count=1, batch_words=4)

Computing transition probabilities:   0%|          | 0/799 [00:00<?, ?it/s]

Generating walks (CPU: 6): 100%|██████████| 66/66 [00:12<00:00,  5.43it/s]


KeyboardInterrupt: 

In [ ]:
# node2vec = Node2Vec(UG, dimensions=64, walk_length=30, num_walks=200, workers=4)
# model = node2vec.fit(window=10, min_count=1)
node_embeddings = embeddings.wv

X = []
for node_id in node_embeddings.index_to_key:
    embedding = node_embeddings[node_id]
    X.append(embedding)

clustering = DBSCAN(eps=0.5, min_samples=5, metric='euclidean')
clustering.fit(X)

clusters = {}
for i, label in enumerate(clustering.labels_):
    if label not in clusters:
        clusters[label] = []
    clusters[label].append(node_embeddings.index2entity[i])

circular_trading = []
for cluster in clusters.values():
    if len(cluster) >= 3:
        subgraph = DG.subgraph(cluster)
        cycles = list(nx.simple_cycles(subgraph))
        if cycles:
            circular_trading.append(cycles)

In [ ]:
undirected_graph[1309][1955]

AtlasView({0: {'Value': 399866.0}, 1: {'Value': 286499.0}, 2: {'Value': 467280.0}, 3: {'Value': 460200.0}, 4: {'Value': 381730.0}, 5: {'Value': 362850.0}})